In [1]:
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
import json
import requests

from functools import reduce
import yfinance as yf

import datetime as dt
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta

# Import PlotLy Dependencies
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import tree

In [3]:
# Run the following Jupyter Notebooks first
%run data_source.ipynb
%run inflation.ipynb
%run productivity.ipynb
%run jobs.ipynb
%run housing.ipynb

Stored 'inflation_change_df' (DataFrame)
Stored 'productivity_index_monthly_data' (DataFrame)
Stored 'gdp_monthly_data' (DataFrame)
Avg. Median Income Change:-  0.6893301036556077
New Datapoint:-  75094.10239130635
Stored 'jobs_index_change_df' (DataFrame)
Stored 'housing_monthly_data' (DataFrame)


# Inflation

In [4]:
# View Inflation Data
inflation_change_df

,Date,Overall Inflation Index,Monthly Change in Inflation Index (%),Inflation Change in Velocity (basis points)
0,1959-01-01,0.000686,NaN,NaN
1,1959-02-01,0.000732,6.262029,NaN
2,1959-03-01,0.000749,2.376992,-388.503682
3,1959-04-01,0.000913,17.897456,1552.046458
4,1959-05-01,0.001040,12.268630,-562.882680
...,...,...,...,...
773,2023-06-01,0.947398,0.226790,65.911579
774,2023-07-01,0.950052,0.279385,5.259543
775,2023-08-01,0.959447,0.979231,69.984609
776,2023-09-01,0.964861,0.561064,-41.816751


# Productivity

In [5]:
# View Productivity Data. Notice that the data is lagging. For this reason, we will use only the GDP number for productivity.
productivity_index_monthly_data

,Date,Overall Productivity Index,Change in Productivity Index (%),Productivity Change in Velocity (basis points)
0,1966-01-01,0.119600,NaN,NaN
1,1966-02-01,0.119600,NaN,NaN
2,1966-03-01,0.119600,NaN,NaN
3,1966-04-01,0.121922,1.904611,NaN
4,1966-05-01,0.121922,1.904611,NaN
...,...,...,...,...
688,2023-05-01,0.742139,0.479013,-79.665286
689,2023-06-01,0.742139,0.479013,-79.665286
690,2023-07-01,0.760044,2.355768,187.675499
691,2023-08-01,0.760044,2.355768,187.675499


In [6]:
# Decided to just use the GDP data only for productivity
gdp_monthly_data

,Date,Overall Productivity Index,Change in GDP (%),GDP Change in Velocity (basis points)
0,1947-01-01,243.164,NaN,NaN
1,1947-02-01,243.164,NaN,NaN
2,1947-03-01,243.164,NaN,NaN
3,1947-04-01,245.968,1.139986,NaN
4,1947-05-01,245.968,1.139986,NaN
...,...,...,...,...
916,2023-05-01,27063.012,0.921594,-58.956481
917,2023-06-01,27063.012,0.921594,-58.956481
918,2023-07-01,27644.463,2.103318,118.172447
919,2023-08-01,27644.463,2.103318,118.172447


# Jobs

In [7]:
# View Jobs Data
jobs_index_change_df

,Date,Overall Jobs Index,Monthly Change in Jobs Index (%),Jobs Index Change in Velocity (basis points)
0,1984-01-01,0.454794,NaN,NaN
1,1984-02-01,0.471976,3.640625,NaN
2,1984-03-01,0.471976,0.000000,-364.062500
3,1984-04-01,0.484735,2.631979,263.197931
4,1984-05-01,0.506342,4.267405,163.542529
...,...,...,...,...
473,2023-06-01,0.765919,0.577709,232.090468
474,2023-07-01,0.770344,0.574390,-0.331830
475,2023-08-01,0.765403,-0.645543,-121.993316
476,2023-09-01,0.765403,0.000000,64.554294


# Housing

In [8]:
# View Housing Data. Notice that the data is lagging. For this reason, we will use a 12 month moving average to bring the data to latest date that will align with other dataframes.
housing_monthly_data

,Date,Overall Housing Index,Change in Overall Housing Index (%),Housing Change in Velocity (basis points)
0,1987-04-01,0.526798,NaN,NaN
1,1987-05-01,0.526798,NaN,NaN
2,1987-06-01,0.526798,NaN,NaN
3,1987-07-01,0.525306,-0.284153,NaN
4,1987-08-01,0.525306,-0.284153,NaN
...,...,...,...,...
430,2023-02-01,0.926477,-0.877472,215.180349
431,2023-03-01,0.926477,-0.877472,215.180349
432,2023-04-01,0.969956,4.482617,536.008947
433,2023-05-01,0.969956,4.482617,536.008947


In [9]:
updated_housing_monthly_data = pd.DataFrame()

dates_list = list(housing_monthly_data["Date"])
i_list = list(housing_monthly_data["Overall Housing Index"])[-12:]

change_list = list(housing_monthly_data["Change in Overall Housing Index (%)"])

# last_change = list(housing_monthly_data["Housing Change in Velocity (basis points)"])[-1]
velocity_list = []

for i in range(5):
    # New Dates
    dates_list.append(dates_list[-1] + pd.DateOffset(months=1))
    
    # Index Averages to add
    avg = sum(i_list[-12:]) / 12
    
    # Change (%) 
    change = ((avg - i_list[-1]) / i_list[-1]) * 100  
    i_list.append(avg)
    
    # Velocity
    velocity = (change - change_list[-1]) * 100
    
    change_list.append(change)
    velocity_list.append(velocity)
        

fabricated_index_list = i_list[-5:]

updated_housing_monthly_data["Date"] = dates_list

indexes = list(housing_monthly_data["Overall Housing Index"]) + fabricated_index_list
updated_housing_monthly_data["Overall Housing Index"] = indexes

changes = list(housing_monthly_data["Change in Overall Housing Index (%)"]) + change_list

updated_housing_monthly_data["Change in Overall Housing Index (%)"] = change_list

      
velocities = list(housing_monthly_data["Housing Change in Velocity (basis points)"]) + velocity_list
updated_housing_monthly_data["Housing Change in Velocity (basis points)"] = velocities


updated_housing_monthly_data.tail(12)

,Date,Overall Housing Index,Change in Overall Housing Index (%),Housing Change in Velocity (basis points)
428,2022-12-01,0.934606,-3.029276,-63.528110
429,2023-01-01,0.926477,-0.877472,215.180349
430,2023-02-01,0.926477,-0.877472,215.180349
431,2023-03-01,0.926477,-0.877472,215.180349
432,2023-04-01,0.969956,4.482617,536.008947
433,2023-05-01,0.969956,4.482617,536.008947
434,2023-06-01,0.969956,4.482617,536.008947
435,2023-07-01,0.948489,-2.213175,-669.579182
436,2023-08-01,0.947287,-0.126770,208.640488
437,2023-09-01,0.945984,-0.137508,-1.073848


# Interest Rate

In [10]:
# GS10 - Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity. This will be used to gauge rates
interest_rate_df = pd.DataFrame(fred_api_function("GS10"))

# Rename the 'Value' column to '10-Year Real Interest Rate'
interest_rate_df.rename(columns={'Value': '10-Year Treasury Yield'}, inplace=True)

interest_rate_df["10-Yr Yield (basis points)"] = interest_rate_df["10-Year Treasury Yield"] * 100

interest_rate_df

,Date,10-Year Treasury Yield,10-Yr Yield (basis points)
0,1953-04-01,2.83,283.0
1,1953-05-01,3.05,305.0
2,1953-06-01,3.11,311.0
3,1953-07-01,2.93,293.0
4,1953-08-01,2.95,295.0
...,...,...,...
843,2023-07-01,3.90,390.0
844,2023-08-01,4.17,417.0
845,2023-09-01,4.38,438.0
846,2023-10-01,4.80,480.0


In [11]:
# Convert Dates to the appropriate data type
inflation_change_df["Date"] = pd.to_datetime(inflation_change_df["Date"])
gdp_monthly_data["Date"] = pd.to_datetime(gdp_monthly_data["Date"])
jobs_index_change_df["Date"] = pd.to_datetime(jobs_index_change_df["Date"])
updated_housing_monthly_data["Date"] = pd.to_datetime(updated_housing_monthly_data["Date"])
interest_rate_df["Date"] = pd.to_datetime(interest_rate_df["Date"])

In [12]:
# Merge the DataFrames
combined_df = pd.DataFrame()

# Merge Inflation and Productivity DataFrames on the 'Date' column
combined_df = pd.merge(inflation_change_df, gdp_monthly_data, on='Date')

# Merge Combined DataFrames to Jobs DataFrame on the 'Date' column
combined_df = pd.merge(combined_df, jobs_index_change_df, on='Date')

# Merge Combined DataFrames to Housing DataFrame on the 'Date' column
combined_df = pd.merge(combined_df, updated_housing_monthly_data, on='Date')

# Merge Combined DataFrames to Interest Rate DataFrame on the 'Date' column
combined_df = pd.merge(combined_df, interest_rate_df, on='Date')

combined_df

,Date,Overall Inflation Index,Monthly Change in Inflation Index (%),Inflation Change in Velocity (basis points),Overall Productivity Index,Change in GDP (%),GDP Change in Velocity (basis points),Overall Jobs Index,Monthly Change in Jobs Index (%),Jobs Index Change in Velocity (basis points),Overall Housing Index,Change in Overall Housing Index (%),Housing Change in Velocity (basis points),10-Year Treasury Yield,10-Yr Yield (basis points)
0,1987-04-01,0.247756,0.837615,45.520270,4806.160,1.747840,38.132464,0.629590,1.446604,144.660389,0.526798,NaN,NaN,8.02,802.0
1,1987-05-01,0.249359,0.642805,-19.480951,4806.160,1.747840,38.132464,0.642090,1.946768,50.016450,0.526798,NaN,NaN,8.61,861.0
2,1987-06-01,0.250934,0.627828,-1.497712,4806.160,1.747840,38.132464,0.638181,-0.612452,-255.922056,0.526798,NaN,NaN,8.40,840.0
3,1987-07-01,0.252414,0.586484,-4.134358,4884.555,1.604957,-14.288342,0.646773,1.328356,194.080855,0.525306,-0.284153,NaN,8.45,845.0
4,1987-08-01,0.254018,0.631408,4.492308,4884.555,1.604957,-14.288342,0.655364,1.310942,-1.741399,0.525306,-0.284153,NaN,8.76,876.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,2023-05-01,0.945249,-0.432326,-67.040986,27063.012,0.921594,-58.956481,0.761494,-1.743196,-231.430598,0.969956,4.482617,536.008947,3.57,357.0
434,2023-06-01,0.947398,0.226790,65.911579,27063.012,0.921594,-58.956481,0.765919,0.577709,232.090468,0.969956,4.482617,536.008947,3.75,375.0
435,2023-07-01,0.950052,0.279385,5.259543,27644.463,2.103318,118.172447,0.770344,0.574390,-0.331830,0.948489,-2.213175,-669.579182,3.90,390.0
436,2023-08-01,0.959447,0.979231,69.984609,27644.463,2.103318,118.172447,0.765403,-0.645543,-121.993316,0.947287,-0.126770,208.640488,4.17,417.0


In [13]:
# Reduce columns of the DataFrame to only the relevant columns

selected_columns = ["Date", "Inflation Change in Velocity (basis points)", "GDP Change in Velocity (basis points)", \
                   "Jobs Index Change in Velocity (basis points)", "Housing Change in Velocity (basis points)", \
                   "10-Yr Yield (basis points)"]


reduced_df = combined_df[selected_columns]

reduced_df

,Date,Inflation Change in Velocity (basis points),GDP Change in Velocity (basis points),Jobs Index Change in Velocity (basis points),Housing Change in Velocity (basis points),10-Yr Yield (basis points)
0,1987-04-01,45.520270,38.132464,144.660389,NaN,802.0
1,1987-05-01,-19.480951,38.132464,50.016450,NaN,861.0
2,1987-06-01,-1.497712,38.132464,-255.922056,NaN,840.0
3,1987-07-01,-4.134358,-14.288342,194.080855,NaN,845.0
4,1987-08-01,4.492308,-14.288342,-1.741399,NaN,876.0
...,...,...,...,...,...,...
433,2023-05-01,-67.040986,-58.956481,-231.430598,536.008947,357.0
434,2023-06-01,65.911579,-58.956481,232.090468,536.008947,375.0
435,2023-07-01,5.259543,118.172447,-0.331830,-669.579182,390.0
436,2023-08-01,69.984609,118.172447,-121.993316,208.640488,417.0


In [14]:
# Lock or Float Historical Lock/Float Best Decisions
yield_list = reduced_df["10-Yr Yield (basis points)"]

lock_yn_list = []

def generate_lock_decisions(yields):
    # Initialize an empty list to store the output
    output = []

    # Iterate through the yields starting from the second element
    for i in range(0, len(yields)):
        try:
            # Check if the current yield is higher than the previous one
            if yields[i] < yields[i + 1]:
                output.append(1)
            else:
                output.append(0)
        except:
            output.append("predict this")

    return output

lock_yn_list = generate_lock_decisions(yield_list)

reduced_df["lock_yn"] = lock_yn_list

# Notice the "predict this" on the last row to remind us to what decision we are trying to determine
reduced_df

C:\Users\deepe\AppData\Local\Temp\ipykernel_7684\428527774.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Inflation Change in Velocity (basis points),GDP Change in Velocity (basis points),Jobs Index Change in Velocity (basis points),Housing Change in Velocity (basis points),10-Yr Yield (basis points),lock_yn
0,1987-04-01,45.520270,38.132464,144.660389,NaN,802.0,1
1,1987-05-01,-19.480951,38.132464,50.016450,NaN,861.0,0
2,1987-06-01,-1.497712,38.132464,-255.922056,NaN,840.0,1
3,1987-07-01,-4.134358,-14.288342,194.080855,NaN,845.0,1
4,1987-08-01,4.492308,-14.288342,-1.741399,NaN,876.0,1
...,...,...,...,...,...,...,...
433,2023-05-01,-67.040986,-58.956481,-231.430598,536.008947,357.0,1
434,2023-06-01,65.911579,-58.956481,232.090468,536.008947,375.0,1
435,2023-07-01,5.259543,118.172447,-0.331830,-669.579182,390.0,1
436,2023-08-01,69.984609,118.172447,-121.993316,208.640488,417.0,1


In [15]:
# # Drop N/As in the top 6 rows
dropped_df = reduced_df.iloc[6:]
dropped_df

,Date,Inflation Change in Velocity (basis points),GDP Change in Velocity (basis points),Jobs Index Change in Velocity (basis points),Housing Change in Velocity (basis points),10-Yr Yield (basis points),lock_yn
6,1987-10-01,35.002992,85.988237,119.636698,87.482221,952.0,0
7,1987-11-01,-13.672016,85.988237,73.594164,87.482221,886.0,1
8,1987-12-01,-2.006025,85.988237,-67.057645,87.482221,899.0,0
9,1988-01-01,39.081656,-117.618938,57.289325,-39.827122,867.0,0
10,1988-02-01,-40.778699,-117.618938,-62.295450,-39.827122,821.0,1
...,...,...,...,...,...,...,...
433,2023-05-01,-67.040986,-58.956481,-231.430598,536.008947,357.0,1
434,2023-06-01,65.911579,-58.956481,232.090468,536.008947,375.0,1
435,2023-07-01,5.259543,118.172447,-0.331830,-669.579182,390.0,1
436,2023-08-01,69.984609,118.172447,-121.993316,208.640488,417.0,1


In [16]:
# Drop Last Row to start building the ML model
dropped_last_row_df = dropped_df.iloc[:-1]
dropped_last_row_df
# Drop first column
dropped_first_column_df = dropped_last_row_df.iloc[:, 1:]
dropped_first_column_df



,Inflation Change in Velocity (basis points),GDP Change in Velocity (basis points),Jobs Index Change in Velocity (basis points),Housing Change in Velocity (basis points),10-Yr Yield (basis points),lock_yn
6,35.002992,85.988237,119.636698,87.482221,952.0,0
7,-13.672016,85.988237,73.594164,87.482221,886.0,1
8,-2.006025,85.988237,-67.057645,87.482221,899.0,0
9,39.081656,-117.618938,57.289325,-39.827122,867.0,0
10,-40.778699,-117.618938,-62.295450,-39.827122,821.0,1
...,...,...,...,...,...,...
432,51.625244,-58.956481,-54.416452,536.008947,346.0,1
433,-67.040986,-58.956481,-231.430598,536.008947,357.0,1
434,65.911579,-58.956481,232.090468,536.008947,375.0,1
435,5.259543,118.172447,-0.331830,-669.579182,390.0,1


In [17]:
# Features Data
features = dropped_first_column_df.iloc[:, :-1]
features_data_matrix = features.values
print(len(features_data_matrix))
# Decision Labels
labels = dropped_first_column_df.iloc[:, -1]
labels_data = labels.values.astype(int)
print(len(labels_data))

431
431


In [18]:
features_data_matrix

array([[ 3.50029920e+01,  8.59882366e+01,  1.19636698e+02,
         8.74822214e+01,  9.52000000e+02],
       [-1.36720159e+01,  8.59882366e+01,  7.35941639e+01,
         8.74822214e+01,  8.86000000e+02],
       [-2.00602538e+00,  8.59882366e+01, -6.70576445e+01,
         8.74822214e+01,  8.99000000e+02],
       ...,
       [ 6.59115785e+01, -5.89564811e+01,  2.32090468e+02,
         5.36008947e+02,  3.75000000e+02],
       [ 5.25954264e+00,  1.18172447e+02, -3.31830125e-01,
        -6.69579182e+02,  3.90000000e+02],
       [ 6.99846092e+01,  1.18172447e+02, -1.21993316e+02,
         2.08640488e+02,  4.17000000e+02]])

In [19]:
labels_data

array([0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,

In [20]:
dropped_first_column_df

,Inflation Change in Velocity (basis points),GDP Change in Velocity (basis points),Jobs Index Change in Velocity (basis points),Housing Change in Velocity (basis points),10-Yr Yield (basis points),lock_yn
6,35.002992,85.988237,119.636698,87.482221,952.0,0
7,-13.672016,85.988237,73.594164,87.482221,886.0,1
8,-2.006025,85.988237,-67.057645,87.482221,899.0,0
9,39.081656,-117.618938,57.289325,-39.827122,867.0,0
10,-40.778699,-117.618938,-62.295450,-39.827122,821.0,1
...,...,...,...,...,...,...
432,51.625244,-58.956481,-54.416452,536.008947,346.0,1
433,-67.040986,-58.956481,-231.430598,536.008947,357.0,1
434,65.911579,-58.956481,232.090468,536.008947,375.0,1
435,5.259543,118.172447,-0.331830,-669.579182,390.0,1


In [21]:
feature_names = list(dropped_first_column_df.columns[:5])
feature_names

['Inflation Change in Velocity (basis points)',
 'GDP Change in Velocity (basis points)',
 'Jobs Index Change in Velocity (basis points)',
 'Housing Change in Velocity (basis points)',
 '10-Yr Yield (basis points)']

# Machine Learning Models

Determine Independent variables (X) and Dependent variables (y). Then split the data into train and test data

In [22]:
from sklearn.model_selection import train_test_split
X = features_data_matrix
y = labels_data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y, shuffle=True, test_size=0.2)
print(X.shape, y.shape)

(431, 5) (431,)


### Decision Tree

In [23]:
from sklearn import tree

classifier = tree.DecisionTreeClassifier()
classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.5172413793103449


In [24]:
fn=feature_names
cn=['Float', 'Lock']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=300)
tree.plot_tree(classifier,
               feature_names = fn, 
               class_names=cn,
               filled = True);fig.savefig('test.png')

In [25]:
# import graphviz
# from sklearn.tree import export_graphviz

# # Visualize the tree
# from IPython.display import display
# display(graphviz.Source(export_graphviz(classifier)))

# # Save as png file
# graph = graphviz.Source(export_graphviz(classifier))
# graph.format = 'png'
# graph.view(filename='tree_image', directory='./')

In [26]:
# Enter the new independent variables to predict decision
new_data = np.array([[-42.802841, 118.172447, 64.554294, -1.073848, 438.0 ]])
new_data

array([[-42.802841, 118.172447,  64.554294,  -1.073848, 438.      ]])

In [27]:
# Predict the decision of the new data point
predictions = classifier.predict(new_data)
print("Classes are either 0 (Float) or 1 (Lock)")
print(f"The new point was classified as: {predictions}")
print("This means that it is predicting that the Treasury yield will go down and to float. Floating is suggested")

Classes are either 0 (Float) or 1 (Lock)
The new point was classified as: [0]
This means that it is predicting that the Treasury yield will go down and to float. Floating is suggested


In [28]:
# How would the model predict the test data
test_predictions = classifier.predict(X_test)
results_df = pd.DataFrame({"Prediction": test_predictions, "Actual": y_test})
results_df

,Prediction,Actual
0,1,1
1,0,0
2,0,1
3,0,1
4,1,1
...,...,...
82,1,0
83,1,0
84,1,1
85,0,1


In [29]:
match_yn = []
for index, row in results_df.iterrows():
    if row[0] == row[1]:
        match_yn.append(1)
    else:
        match_yn.append(0)
        
results_df["match_yn"] = match_yn
results_df

,Prediction,Actual,match_yn
0,1,1,1
1,0,0,1
2,0,1,0
3,0,1,0
4,1,1,1
...,...,...,...
82,1,0,0
83,1,0,0
84,1,1,1
85,0,1,0


In [30]:
# Calculate the percentage of matched
count_results = results_df['match_yn'].value_counts()[1]
percentage = (count_results / len(results_df['match_yn'])) * 100
percentage

51.724137931034484

### Random Forest


###### We can use a random forest classifier to determine how important each feature is to the classify the lock_yn decision

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y, shuffle=True, test_size=0.2)

In [37]:
# How would a random forest classifier score?
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.5517241379310345

In [38]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
features_list = ['Monthly Change in Inflation Index (%)', 'Monthly Change in GDP (%)', 'Monthly Change in Jobs Index (%)', \
            'Monthly Change in Overall Housing Index (%)']

# We can sort the features by their importance
sorted(zip(importances, features_list), reverse=True)

[(0.24790109302605576, 'Monthly Change in Inflation Index (%)'),
 (0.21263702718233077, 'Monthly Change in Jobs Index (%)'),
 (0.16774696020169166, 'Monthly Change in Overall Housing Index (%)'),
 (0.16586236647338748, 'Monthly Change in GDP (%)')]